In [1]:
#IMPORT LIBRARIES
import os
import time
import datetime
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
from torch.optim.lr_scheduler import LambdaLR
from PIL import Image

In [2]:
#OPTIONS
choices = ['ae_photos', 'apple2orange', 'cezanne2photo','cityscapes',
          'facades', 'horse2zebra', 'iphone2dslr_flower', 'maps',
          'monet2photo', 'summer2winter_yosemite', 'ukiyoe2photo','vangogh2photo']
set_dataset_name = choices[5]  
set_dataset_dir = 'cyclegan_data'
set_debugs_dir = f'cyclegan_debugs/{set_dataset_name}'
set_outimages_dir = f'cyclegan_images/{set_dataset_name}'
set_outmodels_dir = f'cyclegan_models/{set_dataset_name}'
set_G_AB_file = f'{set_outmodels_dir}/G_AB' 
set_G_BA_file = f'{set_outmodels_dir}/G_BA' 
set_D_AB_file = f'{set_outmodels_dir}/D_AB' 
set_D_BA_file = f'{set_outmodels_dir}/D_BA' 
set_random_seed = 1
set_epoch_start = 0
set_epoch_decay = 50
set_epoch_end = 100
set_batch_size = 2 ##if run out of memory, choose a smaller number for set_batch_size
set_lr = 0.0002
set_beta1 = 0.5 
set_beta2 = 0.999 
set_num_cpu = 1
set_save_interval = 100 ##if out of storage, choose a larger number for set_save_interval
set_lambda_cyc = 10.0
set_lambda_id = 5.0

In [3]:
#DOWNLOAD DATASET
download_file = set_dataset_name + '.zip'
url = 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/' + download_file
!mkdir -p $set_dataset_dir
%cd $set_dataset_dir
!wget -N $url
!unzip -o $download_file 
%cd -

/home/deep/Documents/vision_cyclegan/cyclegan_data
--2020-10-22 11:36:47--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116867962 (111M) [application/zip]
Saving to: ‘horse2zebra.zip’

horse2zebra.zip     100%[===================>] 111.45M  1.04MB/s    in 2m 2s   

2020-10-22 11:38:49 (937 KB/s) - ‘horse2zebra.zip’ saved [116867962/116867962]

Archive:  horse2zebra.zip
   creating: horse2zebra/
   creating: horse2zebra/trainA/
  inflating: horse2zebra/trainA/n02381460_6223.jpg  
  inflating: horse2zebra/trainA/n02381460_1567.jpg  
  inflating: horse2zebra/trainA/n02381460_3354.jpg  
  inflating: horse2zebra/trainA/n02381460_299.jpg  
  inflating: horse2zebra/trainA/n02381460_3001.jpg  
  inflating: horse2zebra/trainA/n02

  inflating: horse2zebra/trainA/n02381460_525.jpg  
  inflating: horse2zebra/trainA/n02381460_331.jpg  
  inflating: horse2zebra/trainA/n02381460_1979.jpg  
  inflating: horse2zebra/trainA/n02381460_3512.jpg  
  inflating: horse2zebra/trainA/n02381460_5087.jpg  
  inflating: horse2zebra/trainA/n02381460_1402.jpg  
  inflating: horse2zebra/trainA/n02381460_2268.jpg  
  inflating: horse2zebra/trainA/n02381460_1387.jpg  
  inflating: horse2zebra/trainA/n02381460_2016.jpg  
  inflating: horse2zebra/trainA/n02381460_1405.jpg  
  inflating: horse2zebra/trainA/n02381460_2465.jpg  
  inflating: horse2zebra/trainA/n02381460_476.jpg  
  inflating: horse2zebra/trainA/n02381460_1246.jpg  
  inflating: horse2zebra/trainA/n02381460_1003.jpg  
  inflating: horse2zebra/trainA/n02381460_1484.jpg  
  inflating: horse2zebra/trainA/n02381460_2741.jpg  
  inflating: horse2zebra/trainA/n02381460_671.jpg  
  inflating: horse2zebra/trainA/n02381460_1155.jpg  
  inflating: horse2zebra/trainA/n02381460_811.jpg 

  inflating: horse2zebra/trainA/n02381460_8048.jpg  
  inflating: horse2zebra/trainA/n02381460_1191.jpg  
  inflating: horse2zebra/trainA/n02381460_2714.jpg  
  inflating: horse2zebra/trainA/n02381460_1044.jpg  
  inflating: horse2zebra/trainA/n02381460_4665.jpg  
  inflating: horse2zebra/trainA/n02381460_4783.jpg  
  inflating: horse2zebra/trainA/n02381460_703.jpg  
  inflating: horse2zebra/trainA/n02381460_278.jpg  
  inflating: horse2zebra/trainA/n02381460_2596.jpg  
  inflating: horse2zebra/trainA/n02381460_7476.jpg  
  inflating: horse2zebra/trainA/n02381460_7297.jpg  
  inflating: horse2zebra/trainA/n02381460_4516.jpg  
  inflating: horse2zebra/trainA/n02381460_5891.jpg  
  inflating: horse2zebra/trainA/n02381460_7571.jpg  
  inflating: horse2zebra/trainA/n02381460_6403.jpg  
  inflating: horse2zebra/trainA/n02381460_1305.jpg  
  inflating: horse2zebra/trainA/n02381460_1991.jpg  
  inflating: horse2zebra/trainA/n02381460_1798.jpg  
  inflating: horse2zebra/trainA/n02381460_595.jp

  inflating: horse2zebra/trainA/n02381460_384.jpg  
  inflating: horse2zebra/trainA/n02381460_2525.jpg  
  inflating: horse2zebra/trainA/n02381460_624.jpg  
  inflating: horse2zebra/trainA/n02381460_1915.jpg  
  inflating: horse2zebra/trainA/n02381460_5167.jpg  
  inflating: horse2zebra/trainA/n02381460_4263.jpg  
  inflating: horse2zebra/trainA/n02381460_4305.jpg  
  inflating: horse2zebra/trainA/n02381460_3656.jpg  
  inflating: horse2zebra/trainA/n02381460_7756.jpg  
  inflating: horse2zebra/trainA/n02381460_1847.jpg  
  inflating: horse2zebra/trainA/n02381460_743.jpg  
  inflating: horse2zebra/trainA/n02381460_1602.jpg  
  inflating: horse2zebra/trainA/n02381460_2754.jpg  
  inflating: horse2zebra/trainA/n02381460_3829.jpg  
  inflating: horse2zebra/trainA/n02381460_1423.jpg  
  inflating: horse2zebra/trainA/n02381460_2294.jpg  
  inflating: horse2zebra/trainA/n02381460_195.jpg  
  inflating: horse2zebra/trainA/n02381460_4146.jpg  
  inflating: horse2zebra/trainA/n02381460_182.jpg 

  inflating: horse2zebra/trainA/n02381460_4407.jpg  
  inflating: horse2zebra/trainA/n02381460_4916.jpg  
  inflating: horse2zebra/trainA/n02381460_5374.jpg  
  inflating: horse2zebra/trainA/n02381460_178.jpg  
  inflating: horse2zebra/trainA/n02381460_742.jpg  
  inflating: horse2zebra/trainA/n02381460_4657.jpg  
  inflating: horse2zebra/trainA/n02381460_311.jpg  
  inflating: horse2zebra/trainA/n02381460_4743.jpg  
  inflating: horse2zebra/trainA/n02381460_5386.jpg  
  inflating: horse2zebra/trainA/n02381460_4412.jpg  
  inflating: horse2zebra/trainA/n02381460_4751.jpg  
  inflating: horse2zebra/trainA/n02381460_452.jpg  
  inflating: horse2zebra/trainA/n02381460_211.jpg  
  inflating: horse2zebra/trainA/n02381460_3526.jpg  
  inflating: horse2zebra/trainA/n02381460_9184.jpg  
  inflating: horse2zebra/trainA/n02381460_1182.jpg  
  inflating: horse2zebra/trainA/n02381460_128.jpg  
  inflating: horse2zebra/trainA/n02381460_1674.jpg  
  inflating: horse2zebra/trainA/n02381460_4572.jpg  

...

  inflating: horse2zebra/trainB/n02391049_3276.jpg  
  inflating: horse2zebra/trainB/n02391049_909.jpg  
  inflating: horse2zebra/trainB/n02391049_6951.jpg  
  inflating: horse2zebra/trainB/n02391049_2177.jpg  
  inflating: horse2zebra/trainB/n02391049_10158.jpg  
  inflating: horse2zebra/trainB/n02391049_6386.jpg  
  inflating: horse2zebra/trainB/n02391049_614.jpg  
  inflating: horse2zebra/trainB/n02391049_5189.jpg  
  inflating: horse2zebra/trainB/n02391049_2495.jpg  
  inflating: horse2zebra/trainB/n02391049_6918.jpg  
  inflating: horse2zebra/trainB/n02391049_10398.jpg  
  inflating: horse2zebra/trainB/n02391049_1124.jpg  
  inflating: horse2zebra/trainB/n02391049_272.jpg  
  inflating: horse2zebra/trainB/n02391049_7418.jpg  
  inflating: horse2zebra/trainB/n02391049_261.jpg  
  inflating: horse2zebra/trainB/n02391049_745.jpg  
  inflating: horse2zebra/trainB/n02391049_2656.jpg  
  inflating: horse2zebra/trainB/n02391049_119.jpg  
  inflating: horse2zebra/trainB/n02391049_757.jpg 

  inflating: horse2zebra/trainB/n02391049_36.jpg  
  inflating: horse2zebra/trainB/n02391049_9441.jpg  
  inflating: horse2zebra/trainB/n02391049_3087.jpg  
  inflating: horse2zebra/trainB/n02391049_7324.jpg  
  inflating: horse2zebra/trainB/n02391049_1188.jpg  
  inflating: horse2zebra/trainB/n02391049_497.jpg  
  inflating: horse2zebra/trainB/n02391049_9844.jpg  
  inflating: horse2zebra/trainB/n02391049_2111.jpg  
  inflating: horse2zebra/trainB/n02391049_4873.jpg  
  inflating: horse2zebra/trainB/n02391049_541.jpg  
  inflating: horse2zebra/trainB/n02391049_2818.jpg  
  inflating: horse2zebra/trainB/n02391049_626.jpg  
  inflating: horse2zebra/trainB/n02391049_3428.jpg  
  inflating: horse2zebra/trainB/n02391049_6922.jpg  
  inflating: horse2zebra/trainB/n02391049_463.jpg  
  inflating: horse2zebra/trainB/n02391049_2853.jpg  
  inflating: horse2zebra/trainB/n02391049_2825.jpg  
  inflating: horse2zebra/trainB/n02391049_7584.jpg  
  inflating: horse2zebra/trainB/n02391049_167.jpg  


  inflating: horse2zebra/trainB/n02391049_873.jpg  
  inflating: horse2zebra/trainB/n02391049_2597.jpg  
  inflating: horse2zebra/trainB/n02391049_1086.jpg  
  inflating: horse2zebra/trainB/n02391049_2684.jpg  
  inflating: horse2zebra/trainB/n02391049_725.jpg  
  inflating: horse2zebra/trainB/n02391049_8616.jpg  
  inflating: horse2zebra/trainB/n02391049_685.jpg  
  inflating: horse2zebra/trainB/n02391049_5782.jpg  
  inflating: horse2zebra/trainB/n02391049_403.jpg  
  inflating: horse2zebra/trainB/n02391049_601.jpg  
  inflating: horse2zebra/trainB/n02391049_1846.jpg  
  inflating: horse2zebra/trainB/n02391049_6664.jpg  
  inflating: horse2zebra/trainB/n02391049_715.jpg  
  inflating: horse2zebra/trainB/n02391049_483.jpg  
  inflating: horse2zebra/trainB/n02391049_2959.jpg  
  inflating: horse2zebra/trainB/n02391049_9533.jpg  
  inflating: horse2zebra/trainB/n02391049_5789.jpg  
  inflating: horse2zebra/trainB/n02391049_6629.jpg  
  inflating: horse2zebra/trainB/n02391049_3513.jpg  


  inflating: horse2zebra/trainB/n02391049_4445.jpg  
  inflating: horse2zebra/trainB/n02391049_3314.jpg  
  inflating: horse2zebra/trainB/n02391049_3232.jpg  
  inflating: horse2zebra/trainB/n02391049_10122.jpg  
  inflating: horse2zebra/trainB/n02391049_847.jpg  
  inflating: horse2zebra/trainB/n02391049_4584.jpg  
  inflating: horse2zebra/trainB/n02391049_10239.jpg  
  inflating: horse2zebra/trainB/n02391049_3088.jpg  
  inflating: horse2zebra/trainB/n02391049_1495.jpg  
  inflating: horse2zebra/trainB/n02391049_3158.jpg  
  inflating: horse2zebra/trainB/n02391049_969.jpg  
  inflating: horse2zebra/trainB/n02391049_461.jpg  
  inflating: horse2zebra/trainB/n02391049_8158.jpg  
  inflating: horse2zebra/trainB/n02391049_9303.jpg  
  inflating: horse2zebra/trainB/n02391049_9016.jpg  
  inflating: horse2zebra/trainB/n02391049_3149.jpg  
  inflating: horse2zebra/trainB/n02391049_2286.jpg  
  inflating: horse2zebra/trainB/n02391049_9331.jpg  
  inflating: horse2zebra/trainB/n02391049_3262.

  inflating: horse2zebra/testA/n02381460_3120.jpg  
  inflating: horse2zebra/testA/n02381460_2050.jpg  
  inflating: horse2zebra/testA/n02381460_2150.jpg  
  inflating: horse2zebra/testA/n02381460_5090.jpg  
  inflating: horse2zebra/testA/n02381460_1750.jpg  
  inflating: horse2zebra/testA/n02381460_6300.jpg  
  inflating: horse2zebra/testA/n02381460_7190.jpg  
  inflating: horse2zebra/testA/n02381460_3660.jpg  
  inflating: horse2zebra/testA/n02381460_5500.jpg  
  inflating: horse2zebra/testA/n02381460_1100.jpg  
  inflating: horse2zebra/testA/n02381460_2540.jpg  
  inflating: horse2zebra/testA/n02381460_6640.jpg  
  inflating: horse2zebra/testA/n02381460_2890.jpg  
  inflating: horse2zebra/testA/n02381460_4630.jpg  
  inflating: horse2zebra/testA/n02381460_7140.jpg  
  inflating: horse2zebra/testA/n02381460_7250.jpg  
  inflating: horse2zebra/testA/n02381460_2120.jpg  
  inflating: horse2zebra/testA/n02381460_4160.jpg  
  inflating: horse2zebra/testA/n02381460_900.jpg  
  inflating: 

In [4]:
#DATASET
##define custom dataset
class CustomDataset(Dataset):
    def __init__(self, root, train=True, transforms=None):
        self.train = train
        if train:
            path_A, dirs_A, file_names_A = next(os.walk(root + '/trainA/'))
            path_B, dirs_B, file_names_B = next(os.walk(root + '/trainB/'))
        else:
            path_A, dirs_A, file_names_A  = next(os.walk(root + '/testA/'))
            path_B, dirs_B, file_names_B = next(os.walk(root + '/testB/'))
        self.abs_paths_A = [path_A + name for name in file_names_A]
        self.abs_paths_B = [path_B + name for name in file_names_B]
        ##transform
        self.transforms = transforms
        
    def convert_rgb(self, image):
        if image.mode != "RGB":
            image = Image.new("RGB", image.size)
            image.paste(image)
        return image

    def __getitem__(self, index):
        ###pair (image_A, image_B) 
        index_A = random.randint(0, len(self.abs_paths_A) - 1)
        index_B = random.randint(0, len(self.abs_paths_B) - 1)
        image_A = Image.open(self.abs_paths_A[index_A])
        image_B = Image.open(self.abs_paths_B[index_B])
        ###Convert grayscale images to rgb
        image_A = self.convert_rgb(image_A)
        image_B = self.convert_rgb(image_B)
        ###transform to tensor    
        if self.transforms is not None:
            image_A = self.transforms(image_A)
            image_B = self.transforms(image_B)
        return {"A": image_A, "B": image_B }

    def __len__(self):
        return max(len(self.abs_paths_A ), len(self.abs_paths_B))


##image transformations
custom_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
    transforms.Resize((286,286)),
    transforms.RandomCrop((256,256)),
    transforms.ToTensor(),###to range [0, 1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])###to range [-1 1]
    
##create dataloader    
train_dataset = CustomDataset(f'{set_dataset_dir}/{set_dataset_name}', True, custom_transform)
val_dataset = CustomDataset(f'{set_dataset_dir}/{set_dataset_name}', False, custom_transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=set_batch_size, shuffle=True, num_workers=set_num_cpu)
val_loader = DataLoader(dataset=val_dataset, batch_size=5, shuffle=True, num_workers=set_num_cpu)
save_condition = set_save_interval <= len(train_loader)
assert save_condition, f'''
The set_save_interval must be less than or equal to total number batches in one epoch of train dataset,
so it must lie between [1, {len(train_loader)})
'''


In [5]:
#DEFINE BASE MODELS
class ConvolutionalBlock(nn.Module):
    def __init__(self, kernel_name, in_channels, out_channels, kernel_size, stride, padding, activation_name=None):
        super(ConvolutionalBlock, self).__init__()
        kernel_layer = getattr(nn, kernel_name)
        self.covolutional_block = nn.Sequential(
            kernel_layer(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        )
        if activation_name is not None:
            activation = getattr(nn, activation_name) 
            self.covolutional_block.add_module('activation', activation())
                
    def forward(self, x):
        return self.covolutional_block(x)
##--------------------------------------------------------------------------------------------------------------   
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.residual = nn.Sequential(
            ConvolutionalBlock('Conv2d', in_channels, in_channels, 3, 1, 1, 'ReLU'),
            ConvolutionalBlock('Conv2d', in_channels, in_channels, 3, 1, 1, 'ReLU'))

    def forward(self, x):
        shortcut = x ###shortcut path
        out = self.residual(x) ###main path
        out += shortcut ###gather path
        return out
##--------------------------------------------------------------------------------------------------------------
class MultipleResiduals(nn.Module):
    def __init__(self, in_channels, num_repeat):
        super(MultipleResiduals, self).__init__()
        index = list(range(num_repeat))
        self.multiple_residuals = nn.Sequential()
        for i in range(num_repeat):
            self.multiple_residuals.add_module(f'{index[i]}th_multiple_residuals', ResidualBlock(in_channels))
    
    def forward(self, x):
        return self.multiple_residuals(x)    

<img src="static/depict/cyclegan_generator.png" style="width:70%"/>

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        ###generator is similar to autoencoder
        self.generator = nn.Sequential(
            ###encoder part
            ### formular to calculate padding for Conv2d
            ### at https://sebastianraschka.com/pdf/lecture-notes/stat479ss19/L13_intro-cnn-part2_slides.pdf
                ### (i - k + 2*p)/s + 1 = o round floor
                ### => p = (s(o-1) - i + k)/2 ###round up
                ### if s=1, o=i then p = (1(i-1) - i + k)/2 = (k-1)/2
            ###kernel_name, in_channels, out_channels, kernel_size, stride, padding, activation_name
            ConvolutionalBlock('Conv2d', 3, 64, 7, 1, 3, 'ReLU'), ### i
            ConvolutionalBlock('Conv2d', 64, 128, 3, 2, 1, 'ReLU'), ### i/2
            ConvolutionalBlock('Conv2d', 128, 256, 3, 2, 1, 'ReLU'), ### i/4
            
            ###transformer part
            MultipleResiduals(256, 6), ### i/4
            
            ###decoder part
            ### calculate padding for ConvTranspose2d:
                ###o = s(n-1) + k - 2p
                ###if s=2 then output = 2(n-1) + k - 2p = 2n - 2 + k -2p
            ###kernel_name, in_channels, out_channels, kernel_size, stride, padding, activation_name
            ConvolutionalBlock('ConvTranspose2d', 256, 128, 2, 2, 0, 'ReLU'), ### i/2
            ConvolutionalBlock('ConvTranspose2d', 128, 64, 2, 2, 0, 'ReLU'), ### i
            ConvolutionalBlock('Conv2d', 64, 3, 7, 1, 3, 'Tanh') ### i
        )

    def forward(self, x):
        return self.generator(x) ###output size(set_batch_size, 3, 256, 256)

<img src="static/depict/cyclegan_discriminator.png" style="width:70%"/>

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.discriminator = nn.Sequential(
            ###kernel_name, in_channels, out_channels, kernel_size, stride, padding, activation_name
            ConvolutionalBlock('Conv2d', 3, 64, 4, 2, 1, 'LeakyReLU'), ### i/2
            ConvolutionalBlock('Conv2d', 64, 128, 4, 2, 1, 'LeakyReLU'), ### i/4
            ConvolutionalBlock('Conv2d', 128, 256, 4, 2, 1, 'LeakyReLU'), ### i/8
            ConvolutionalBlock('Conv2d', 256, 512, 4, 2, 1, 'LeakyReLU'), ### i/16
            nn.ZeroPad2d((1, 0, 1, 0)),###reference at https://pytorch.org/docs/master/generated/torch.nn.ZeroPad2d.html
            ConvolutionalBlock('Conv2d', 512, 1, 4, 1, 1, 'Sigmoid'), ### i/16
        )
        
    def forward(self, x):
        return self.discriminator(x) ###output size(set_batch_size, 1, 16, 16)

In [8]:
#SETUP MODELS
## initialize models
torch.manual_seed(set_random_seed)
G_AB = Generator()
G_BA = Generator()
D_A = Discriminator()
D_B = Discriminator()
    

##device
###one gpu or cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
G_AB = G_AB.to(device)
G_BA = G_BA.to(device)
D_A = D_A.to(device)
D_B = D_B.to(device)
parallel=False
### multiple gpus
if device.type != 'cpu' and torch.cuda.device_count() > 1:
    parallel = True
    G_AB = nn.DataParallel(G_AB)
    G_BA = nn.DataParallel(G_BA)
    D_A = nn.DataParallel(D_A)
    D_B = nn.DataParallel(D_B)
    

## load pretrained model weights if you train model in several times
if set_epoch_start > 0:  
    ###in order to use pretrained model weight, you must rename the last weight G_AB@number1_number2.pt to G_AB.pt 
    set_G_AB_file = f'{set_outmodels_dir}/G_AB.pt' 
    ###in order to use pretrained model weight, you must rename the last weight G_BA@number1_number2.pt to G_BA.pt 
    set_G_BA_file = f'{set_outmodels_dir}/G_BA.pt' 
    ###in order to use pretrained model weight, you must rename the last weight D_AB@number1_number2.pt to D_AB.pt 
    set_D_AB_file = f'{set_outmodels_dir}/D_AB.pt' 
    ###in order to use pretrained model weight, you must rename the last weight D_BA@number1_number2.pt to D_BA.pt 
    set_D_BA_file = f'{set_outmodels_dir}/D_BA.pt' 
    
    models = [G_AB, G_BA, D_A, D_B]
    model_files = [set_G_AB_file, set_G_BA_file, set_D_AB_file, set_D_BA_file]
    for f, m in zip(model_files, models):
        if parallel:
            m.module.load_state_dict(torch.load(f))
        else:
            m.load_state_dict(torch.load(f))             

##optimizers
optim_G_AB = torch.optim.Adam(G_AB.parameters(), lr=set_lr, betas=(set_beta1, set_beta2))
optim_G_BA = torch.optim.Adam(G_BA.parameters(), lr=set_lr, betas=(set_beta1, set_beta2))
optim_D_A = torch.optim.Adam(D_A.parameters(), lr=set_lr, betas=(set_beta1, set_beta2))
optim_D_B = torch.optim.Adam(D_B.parameters(), lr=set_lr, betas=(set_beta1, set_beta2))


##learing rate scheduler
condition_1 = set_epoch_decay > set_epoch_start
condition_2 = set_epoch_decay < set_epoch_end
assert condition_1 and  condition_2, 'The set_epoch_decay value must lie between (set_epoch_start, set_epoch_end)'
lambda_func = lambda epoch: 1 - max(0, epoch + set_epoch_start - set_epoch_decay ) / (set_epoch_end - set_epoch_decay)
scheduler_G_AB = LambdaLR(optim_G_AB, lr_lambda=lambda_func)
scheduler_G_BA = LambdaLR(optim_G_BA, lr_lambda=lambda_func)
scheduler_D_A = LambdaLR(optim_D_A, lr_lambda=lambda_func)
scheduler_D_B = LambdaLR(optim_D_B, lr_lambda=lambda_func)

##generate cyclegan image from val_loader input
def sample_images(image_name):
    batch = next(iter(val_loader))
    with torch.no_grad():
        real_A = batch["A"].to(device)
        real_B = batch["B"].to(device)
        fake_A = G_BA(real_B)
        fake_B = G_AB(real_A)
    ### Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)
    real_B = make_grid(real_B, nrow=5, normalize=True)
    fake_A = make_grid(fake_A, nrow=5, normalize=True)
    fake_B = make_grid(fake_B, nrow=5, normalize=True)
    ### Arange images along y-axis
    image = torch.cat((real_A, fake_B, real_B, fake_A), 1)
    ###save image
    if os.path.exists(set_outimages_dir) == False:
        os.makedirs(set_outimages_dir)
    set_outimage_file = f'{set_outimages_dir}/{image_name}.png'
    save_image(image, set_outimage_file, normalize=False)
    return image
device

device(type='cuda', index=0)

In [9]:
# TRAINING 
print('Please wait for training ...', end="")
for epoch in range(set_epoch_start, set_epoch_end):
    for batch_idx, batch in enumerate(train_loader):
        #starting time for every batch
        time_start = time.time()
        ##Inputs And Ground Truths
        real_A = batch["A"].to(device)
        real_B = batch["B"].to(device)
        p_size = (real_A.size(0), 1, 16, 16) 
        valid = torch.ones(p_size).float().to(device)
        fake = torch.zeros(p_size).float().to(device)
        
        ##Train Generators
        ###start from input A
        fake_B = G_AB(real_A)
        cycle_A = G_BA(fake_B)
        id_A = G_BA(real_A)
        ###start from input B
        fake_A = G_BA(real_B)
        cycle_B = G_AB(fake_A)
        id_B = G_AB(real_B)
        ###loss gan from input A
        loss_gan_AB = F.l1_loss(D_B(fake_B), valid)
        loss_cycle_A = F.l1_loss(cycle_A, real_A)
        loss_id_A = F.l1_loss(id_A, real_A)
        ###loss gan from input B
        loss_gan_BA = F.l1_loss(D_A(fake_A), valid)
        loss_cycle_B = F.l1_loss(cycle_B, real_B)
        loss_id_B = F.l1_loss(id_B, real_B)
        ###total loss_generator
        loss_gan = 0.5 * (loss_gan_AB + loss_gan_BA)
        loss_cycle = 0.5 * (loss_cycle_A + loss_cycle_B)
        loss_identity = 0.5 * (loss_id_A + loss_id_B)
        loss_G = loss_gan + set_lambda_cyc * loss_cycle + set_lambda_id * loss_identity
        ###backward and update gradient
        optim_G_AB.zero_grad()
        optim_G_BA.zero_grad()
        loss_G.backward()
        optim_G_AB.step()
        optim_G_BA.step()
        

        ##Train Discriminator
        ###loss discriminator from input A
        p_real_A = D_A(real_A)
        p_fake_A = D_A(fake_A.detach())
        loss_real_A = F.mse_loss(p_real_A, valid)
        loss_fake_A = F.mse_loss(p_fake_A, fake)
        ###loss discriminator from input B
        p_real_B = D_B(real_B)
        p_fake_B = D_B(fake_B.detach())
        loss_real_B = F.mse_loss(p_real_B, valid)
        loss_fake_B = F.mse_loss(p_fake_B, fake)
        with torch.no_grad():
            mp_fake_A = p_fake_A.mean().item()####it is the mean of probabilities in p_fake_A
            mp_real_A = p_real_A.mean().item()####it is the mean of probabilities in p_real_A
            mp_fake_B = p_fake_B.mean().item()####it is the mean of probabilities in p_fake_B
            mp_real_B = p_real_B.mean().item()####it is the mean of probabilities in p_real_B
        ##total loss discrimnator
        loss_D_A = 0.5 * (loss_real_A + loss_fake_A)
        loss_D_B = 0.5 * (loss_real_B + loss_fake_B)
        loss_D = 0.5 * (loss_D_A + loss_D_B)
        ##backward and update gradient
        optim_D_A.zero_grad()
        optim_D_B.zero_grad()
        loss_D.backward()
        optim_D_A.step()
        optim_D_B.step()
        
            
        ##for every set_save_interval
        if batch_idx % set_save_interval == 0:
            ### calulate time remain
            if (epoch == set_epoch_start and batch_idx == 0):
                print('...')
                continue
            batches_per_epoch = len(train_loader)
            batches_total = (set_epoch_end - set_epoch_start) * batches_per_epoch
            batches_complete = (epoch - set_epoch_start) * batches_per_epoch + batch_idx
            batches_remain = batches_total - batches_complete
            time_remain = datetime.timedelta(seconds=batches_remain * (time.time() - time_start))
            time_remain = str(time_remain).split(".")[0] ###remove microsecond part
            ###log text
            print('P(real_A): %.10f' % (mp_real_A))
            print('P(fake_A): %.10f' % (mp_fake_A))
            print('P(real_B): %.10f' % (mp_real_B))
            print('P(fake_B): %.10f' % (mp_fake_B))
            print('Epoch:%03d/%03d | Batch:%03d/%03d | D:%.3f | G:%.3f | adv:%.3f | cyc:%.3f | id:%.3f | remain: %s' 
                   %(epoch+1, set_epoch_end, batch_idx, batches_per_epoch,
                   loss_D, loss_G, loss_gan, loss_cycle, loss_identity,
                   time_remain))
            print('======================================================================================================')
            ###save images
            image_name = f'{epoch+1}_{batch_idx}'
            sample_images(image_name)
            ###save models
            models = [G_AB, G_BA, D_A, D_B]
            files = [set_G_AB_file, set_G_BA_file, set_D_AB_file, set_D_BA_file]
            for file, model in zip(files, models):
                if not os.path.exists(set_outmodels_dir):
                    os.makedirs(set_outmodels_dir)
                if parallel:
                    torch.save(model.module.state_dict(), file + f'@{epoch+1}_{batch_idx}.pt')##save weight model when running in parallel
                else:
                    torch.save(model.state_dict(), file + f'@{epoch+1}_{batch_idx}.pt')##save weight model when running without parallel
    ##schedule learning rate
    scheduler_G_AB.step()
    scheduler_G_BA.step()        
    scheduler_D_A.step()
    scheduler_D_B.step()  
    

Please wait for training ......
P(real_A): 0.5676993728
P(fake_A): 0.2921300828
P(real_B): 0.9730014205
P(fake_B): 0.2432948947
Epoch:001/100 | Batch:100/667 | D:0.108 | G:4.518 | adv:0.732 | cyc:0.271 | id:0.215 | remain: 4:25:23
P(real_A): 0.7997951508
P(fake_A): 0.6797287464
P(real_B): 0.9884634614
P(fake_B): 0.3415049016
Epoch:001/100 | Batch:200/667 | D:0.189 | G:3.997 | adv:0.489 | cyc:0.245 | id:0.211 | remain: 4:43:11
P(real_A): 0.4802969098
P(fake_A): 0.4779536724
P(real_B): 0.5168989897
P(fake_B): 0.4914301336
Epoch:001/100 | Batch:300/667 | D:0.261 | G:2.939 | adv:0.515 | cyc:0.163 | id:0.158 | remain: 4:44:53
P(real_A): 0.6904824972
P(fake_A): 0.4281533360
P(real_B): 0.8387335539
P(fake_B): 0.3244273067
Epoch:001/100 | Batch:400/667 | D:0.154 | G:4.085 | adv:0.624 | cyc:0.246 | id:0.201 | remain: 4:46:24
P(real_A): 0.4742887020
P(fake_A): 0.3128874302
P(real_B): 0.5679771900
P(fake_B): 0.3347456455
Epoch:001/100 | Batch:500/667 | D:0.202 | G:2.616 | adv:0.676 | cyc:0.136 | 

P(real_A): 0.5514388084
P(fake_A): 0.4635670781
P(real_B): 0.7752909064
P(fake_B): 0.3713971376
Epoch:005/100 | Batch:100/667 | D:0.172 | G:1.731 | adv:0.583 | cyc:0.081 | id:0.067 | remain: 4:09:15
P(real_A): 0.8869307637
P(fake_A): 0.3281141520
P(real_B): 0.6913442612
P(fake_B): 0.3716279268
Epoch:005/100 | Batch:200/667 | D:0.112 | G:1.746 | adv:0.650 | cyc:0.076 | id:0.067 | remain: 4:05:11
P(real_A): 0.6998137236
P(fake_A): 0.2772536874
P(real_B): 0.8908179998
P(fake_B): 0.3361240327
Epoch:005/100 | Batch:300/667 | D:0.099 | G:1.845 | adv:0.693 | cyc:0.082 | id:0.066 | remain: 4:02:04
P(real_A): 0.4663374424
P(fake_A): 0.4762282372
P(real_B): 0.4652707279
P(fake_B): 0.2963218689
Epoch:005/100 | Batch:400/667 | D:0.245 | G:1.823 | adv:0.614 | cyc:0.085 | id:0.073 | remain: 4:12:50
P(real_A): 0.8014013767
P(fake_A): 0.4387592673
P(real_B): 0.5114586353
P(fake_B): 0.3203271031
Epoch:005/100 | Batch:500/667 | D:0.169 | G:1.681 | adv:0.620 | cyc:0.071 | id:0.069 | remain: 4:09:26
P(rea

P(real_A): 0.6252548695
P(fake_A): 0.5749858618
P(real_B): 0.7844083309
P(fake_B): 0.3880304396
Epoch:009/100 | Batch:100/667 | D:0.200 | G:1.433 | adv:0.518 | cyc:0.064 | id:0.055 | remain: 3:59:46
P(real_A): 0.7719139457
P(fake_A): 0.4189801812
P(real_B): 0.8480603695
P(fake_B): 0.2870134711
Epoch:009/100 | Batch:200/667 | D:0.114 | G:1.792 | adv:0.647 | cyc:0.083 | id:0.064 | remain: 4:10:55
P(real_A): 0.5737833381
P(fake_A): 0.3416564465
P(real_B): 0.7259999514
P(fake_B): 0.2976846099
Epoch:009/100 | Batch:300/667 | D:0.150 | G:1.505 | adv:0.680 | cyc:0.057 | id:0.051 | remain: 4:20:42
P(real_A): 0.5462008119
P(fake_A): 0.4600388408
P(real_B): 0.5047734976
P(fake_B): 0.2817154229
Epoch:009/100 | Batch:400/667 | D:0.206 | G:1.359 | adv:0.629 | cyc:0.051 | id:0.044 | remain: 4:55:53
P(real_A): 0.6855230331
P(fake_A): 0.2363819480
P(real_B): 0.8200898170
P(fake_B): 0.2320836037
Epoch:009/100 | Batch:500/667 | D:0.084 | G:1.420 | adv:0.766 | cyc:0.046 | id:0.040 | remain: 4:04:11
P(rea

P(real_A): 0.6685522199
P(fake_A): 0.4946258068
P(real_B): 0.7226042747
P(fake_B): 0.1752810478
Epoch:013/100 | Batch:100/667 | D:0.153 | G:1.487 | adv:0.665 | cyc:0.058 | id:0.048 | remain: 3:55:51
P(real_A): 0.7923840880
P(fake_A): 0.4349787831
P(real_B): 0.7951707244
P(fake_B): 0.3621949255
Epoch:013/100 | Batch:200/667 | D:0.141 | G:1.507 | adv:0.601 | cyc:0.064 | id:0.052 | remain: 3:49:47
P(real_A): 0.4724704921
P(fake_A): 0.4269478917
P(real_B): 0.6298886538
P(fake_B): 0.2491039783
Epoch:013/100 | Batch:300/667 | D:0.223 | G:1.492 | adv:0.662 | cyc:0.059 | id:0.047 | remain: 3:54:23
P(real_A): 0.6631230116
P(fake_A): 0.3390861154
P(real_B): 0.8496458530
P(fake_B): 0.1950429678
Epoch:013/100 | Batch:400/667 | D:0.115 | G:1.595 | adv:0.733 | cyc:0.064 | id:0.045 | remain: 4:01:12
P(real_A): 0.6001608372
P(fake_A): 0.4433253706
P(real_B): 0.4875696301
P(fake_B): 0.3524867296
Epoch:013/100 | Batch:500/667 | D:0.204 | G:1.366 | adv:0.602 | cyc:0.055 | id:0.042 | remain: 3:48:28
P(rea

P(real_A): 0.8438132405
P(fake_A): 0.4057496190
P(real_B): 0.8619875908
P(fake_B): 0.3952594697
Epoch:017/100 | Batch:100/667 | D:0.134 | G:1.674 | adv:0.599 | cyc:0.081 | id:0.054 | remain: 3:54:18
P(real_A): 0.4730410874
P(fake_A): 0.3120666444
P(real_B): 0.9218736887
P(fake_B): 0.3747313917
Epoch:017/100 | Batch:200/667 | D:0.173 | G:1.356 | adv:0.657 | cyc:0.051 | id:0.037 | remain: 3:48:05
P(real_A): 0.7525029778
P(fake_A): 0.3205111623
P(real_B): 0.7812828422
P(fake_B): 0.3229631782
Epoch:017/100 | Batch:300/667 | D:0.120 | G:1.748 | adv:0.678 | cyc:0.080 | id:0.054 | remain: 3:38:37
P(real_A): 0.6871461868
P(fake_A): 0.5274212956
P(real_B): 0.7250015736
P(fake_B): 0.3723688424
Epoch:017/100 | Batch:400/667 | D:0.209 | G:1.601 | adv:0.550 | cyc:0.078 | id:0.054 | remain: 3:34:07
P(real_A): 0.7576798201
P(fake_A): 0.4811904728
P(real_B): 0.3991919160
P(fake_B): 0.1451003253
Epoch:017/100 | Batch:500/667 | D:0.195 | G:1.486 | adv:0.687 | cyc:0.057 | id:0.045 | remain: 3:38:38
P(rea

...

P(real_A): 0.7905505896
P(fake_A): 0.3795224726
P(real_B): 0.7784868479
P(fake_B): 0.2098153383
Epoch:081/100 | Batch:100/667 | D:0.107 | G:1.097 | adv:0.705 | cyc:0.029 | id:0.021 | remain: 1:08:52
P(real_A): 0.5959773660
P(fake_A): 0.4210850000
P(real_B): 0.8086299896
P(fake_B): 0.2963124812
Epoch:081/100 | Batch:200/667 | D:0.178 | G:1.035 | adv:0.641 | cyc:0.031 | id:0.017 | remain: 0:55:30
P(real_A): 0.6705997586
P(fake_A): 0.3946460783
P(real_B): 0.6734044552
P(fake_B): 0.2528299093
Epoch:081/100 | Batch:300/667 | D:0.186 | G:1.092 | adv:0.676 | cyc:0.030 | id:0.023 | remain: 0:52:02
P(real_A): 0.6790050864
P(fake_A): 0.4160709381
P(real_B): 0.6976387501
P(fake_B): 0.3620610237
Epoch:081/100 | Batch:400/667 | D:0.178 | G:1.062 | adv:0.611 | cyc:0.033 | id:0.024 | remain: 1:02:50
P(real_A): 0.6101825833
P(fake_A): 0.3081931770
P(real_B): 0.7462744713
P(fake_B): 0.1463149190
Epoch:081/100 | Batch:500/667 | D:0.128 | G:1.144 | adv:0.773 | cyc:0.028 | id:0.018 | remain: 0:49:44
P(rea

P(real_A): 0.7618262768
P(fake_A): 0.2440906763
P(real_B): 0.9254326224
P(fake_B): 0.1803974211
Epoch:085/100 | Batch:100/667 | D:0.073 | G:1.302 | adv:0.788 | cyc:0.040 | id:0.023 | remain: 0:44:30
P(real_A): 0.5359328985
P(fake_A): 0.3543645740
P(real_B): 0.7698296905
P(fake_B): 0.2240431458
Epoch:085/100 | Batch:200/667 | D:0.161 | G:1.153 | adv:0.711 | cyc:0.032 | id:0.024 | remain: 0:44:46
P(real_A): 0.6995542645
P(fake_A): 0.1526570618
P(real_B): 0.9178332090
P(fake_B): 0.2429538369
Epoch:085/100 | Batch:300/667 | D:0.068 | G:1.161 | adv:0.802 | cyc:0.027 | id:0.018 | remain: 0:43:50
P(real_A): 0.7239590287
P(fake_A): 0.1836585104
P(real_B): 0.8012866974
P(fake_B): 0.1390591115
Epoch:085/100 | Batch:400/667 | D:0.091 | G:1.370 | adv:0.839 | cyc:0.040 | id:0.026 | remain: 0:42:46
P(real_A): 0.7052769661
P(fake_A): 0.3381155133
P(real_B): 0.8366500139
P(fake_B): 0.3627481759
Epoch:085/100 | Batch:500/667 | D:0.140 | G:1.188 | adv:0.650 | cyc:0.042 | id:0.024 | remain: 0:41:33
P(rea

P(real_A): 0.6485632658
P(fake_A): 0.3667087555
P(real_B): 0.7405523062
P(fake_B): 0.1786517799
Epoch:089/100 | Batch:100/667 | D:0.144 | G:1.201 | adv:0.727 | cyc:0.036 | id:0.023 | remain: 0:30:27
P(real_A): 0.8501707315
P(fake_A): 0.2393106520
P(real_B): 0.8797258139
P(fake_B): 0.2170379162
Epoch:089/100 | Batch:200/667 | D:0.065 | G:1.159 | adv:0.772 | cyc:0.030 | id:0.018 | remain: 0:30:50
P(real_A): 0.5008844137
P(fake_A): 0.1881017983
P(real_B): 0.9230045080
P(fake_B): 0.2514986098
Epoch:089/100 | Batch:300/667 | D:0.136 | G:1.373 | adv:0.780 | cyc:0.044 | id:0.030 | remain: 0:29:42
P(real_A): 0.7412900329
P(fake_A): 0.2777872384
P(real_B): 0.8354475498
P(fake_B): 0.1490996182
Epoch:089/100 | Batch:400/667 | D:0.080 | G:1.232 | adv:0.787 | cyc:0.032 | id:0.025 | remain: 0:36:26
P(real_A): 0.6290119290
P(fake_A): 0.3498898149
P(real_B): 0.8347651362
P(fake_B): 0.2692065239
Epoch:089/100 | Batch:500/667 | D:0.140 | G:1.014 | adv:0.690 | cyc:0.026 | id:0.013 | remain: 0:31:02
P(rea

P(real_A): 0.7167882323
P(fake_A): 0.2850883603
P(real_B): 0.7860919237
P(fake_B): 0.2085711658
Epoch:093/100 | Batch:100/667 | D:0.114 | G:1.153 | adv:0.753 | cyc:0.030 | id:0.021 | remain: 0:21:56
P(real_A): 0.6699175239
P(fake_A): 0.2401015908
P(real_B): 0.7536082268
P(fake_B): 0.2223221660
Epoch:093/100 | Batch:200/667 | D:0.114 | G:1.094 | adv:0.769 | cyc:0.024 | id:0.017 | remain: 0:23:41
P(real_A): 0.5892027617
P(fake_A): 0.3264536560
P(real_B): 0.5904495716
P(fake_B): 0.2837389410
Epoch:093/100 | Batch:300/667 | D:0.200 | G:1.152 | adv:0.695 | cyc:0.032 | id:0.027 | remain: 0:21:41
P(real_A): 0.6445642114
P(fake_A): 0.4052404761
P(real_B): 0.9509375095
P(fake_B): 0.2109185159
Epoch:093/100 | Batch:400/667 | D:0.113 | G:1.127 | adv:0.692 | cyc:0.033 | id:0.020 | remain: 0:18:22
P(real_A): 0.6210944653
P(fake_A): 0.3847116232
P(real_B): 0.8822607398
P(fake_B): 0.1913127303
Epoch:093/100 | Batch:500/667 | D:0.137 | G:1.106 | adv:0.712 | cyc:0.031 | id:0.018 | remain: 0:19:04
P(rea

P(real_A): 0.8250654936
P(fake_A): 0.4468221664
P(real_B): 0.7011839151
P(fake_B): 0.1973125339
Epoch:097/100 | Batch:100/667 | D:0.139 | G:1.109 | adv:0.678 | cyc:0.032 | id:0.023 | remain: 0:09:57
P(real_A): 0.5514855981
P(fake_A): 0.2848233283
P(real_B): 0.8150778413
P(fake_B): 0.1564357877
Epoch:097/100 | Batch:200/667 | D:0.135 | G:1.298 | adv:0.779 | cyc:0.039 | id:0.026 | remain: 0:09:57
P(real_A): 0.5983474851
P(fake_A): 0.2679858804
P(real_B): 0.9647452235
P(fake_B): 0.1879725158
Epoch:097/100 | Batch:300/667 | D:0.102 | G:1.207 | adv:0.772 | cyc:0.034 | id:0.018 | remain: 0:09:13
P(real_A): 0.8316633701
P(fake_A): 0.3660380840
P(real_B): 0.7491886616
P(fake_B): 0.1728419363
Epoch:097/100 | Batch:400/667 | D:0.113 | G:1.044 | adv:0.731 | cyc:0.024 | id:0.015 | remain: 0:08:55
P(real_A): 0.5676398277
P(fake_A): 0.2124568522
P(real_B): 0.9165123701
P(fake_B): 0.2835936546
Epoch:097/100 | Batch:500/667 | D:0.140 | G:1.244 | adv:0.752 | cyc:0.038 | id:0.022 | remain: 0:08:43
P(rea

In [10]:
#RELEASE GPU(s) MEMORY IF USING GPU(s)
del G_AB
del G_BA
del D_A
del D_B
torch.cuda.empty_cache()